In [1]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import ParameterGrid
from transformers import AutoModelForCausalLM, AutoTokenizer
from accelerate import Accelerator
from accelerate.utils import gather_object
from statistics import mean
import transformers
import torch, time, json
import pandas as pd
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:50"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

accelerator = Accelerator()

# load a base model and tokenizer in current directory
access_token = #your token here
model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token, cache_dir='./')
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token, cache_dir='./',
                                             device_map='auto',
                                             torch_dtype=torch.bfloat16)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
def compute_metrics(predictions, references, labels=None, pos_label=1, average="weighted", sample_weight=None, zero_division='warn'):
        f1 = f1_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight
        )
        p = precision_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight,
            zero_division=zero_division
        )
        r = recall_score(
            references, predictions, labels=labels, pos_label=pos_label, average=average, sample_weight=sample_weight,
            zero_division=zero_division
        )
        c = classification_report(
            references, predictions, labels=labels
        )
        print(c)
        return {"f1": float(f1) if f1.size == 1 else f1,
                "precision": float(p) if p.size == 1 else p,
                "recall": float(r) if r.size == 1 else r}

def train_sentence_selection(df, n_neutral, n_positive, n_negative, label, seed):
    # saving column name given PT or MD label
    label = f"{label}_label"
    # randomly selecting 1-2 sentences per label
    neutral_sentences = df[df[label] == "neutral"].sample(n_neutral, replace=False, random_state=seed)
    positive_sentences = df[df[label] == "positive"].sample(n_positive, replace=False, random_state=seed)
    negative_sentences = df[df[label] == "negative"].sample(n_negative, replace=False, random_state=seed)
    all_sentences = pd.concat([neutral_sentences,
                               positive_sentences,
                               negative_sentences], ignore_index = True)
    return all_sentences

### Zero-shot approach
Below is code to evaluate the model with a zero-shot approach.
Update the right dataset and file path.

In [3]:
## zero shot approach
accelerator.wait_for_everyone()
start=time.time()

dataset = "80" #example, evaluating the model on sentences with at least 80% agreement

test = pd.read_csv(f"yourpath/data/test_{dataset}.csv")

# converting test sentences to json format
json_test_sentences = test["language"].to_json()

# creating context prompt

SYSTEM_PROMPT = """
<|start_header_id|>system<|end_header_id|>
You are a patient at a medical center.<|eot_id|>
"""

PROMPTS = []

for sentence in test["language"]:
    
    USER_PROMPT_1 = f"""
    <|start_header_id|>user<|end_header_id|>
    Doctors write lots of clinical notes about you.
    Your task is to analyze the sentiment of a sentence your doctor wrote about you.
    For each sentence, how do you feel reading this description of you?
    Please assign a sentiment score of negative, neutral, or positive for the sentence.
    Only output your sentiment score in JSON format for this sentence:
    {sentence}<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """
    
    PROMPT = [f"""
    {SYSTEM_PROMPT}
    {USER_PROMPT_1}
    """]
    PROMPTS.append(PROMPT)


with accelerator.split_between_processes(PROMPTS) as prompts:
    # store output of generations in dict   
    results=dict(outputs=[])

    # have each GPU do inference, prompt by prompt
    for prompt in prompts:
        prompt_tokenized=tokenizer(prompt, return_tensors="pt").to("cuda")
        output_tokenized = model.generate(**prompt_tokenized, max_new_tokens=10, do_sample=True, temperature=0.001, pad_token_id=tokenizer.eos_token_id)[0]

        # remove prompt from output
        output_tokenized=output_tokenized[len(prompt_tokenized["input_ids"][0]):]

        # store outputs and number of tokens in result{}
        results["outputs"].append( tokenizer.decode(output_tokenized) )
        
    results=[ results ] # transform to list, otherwise gather_object() will not collect correctly

    # collect results from all the GPUs
results_gathered=gather_object(results)
results = results_gathered[0]['outputs']
parsed_results = []
for label in results:
    label = label.strip().replace('<|eot_id|>', '')
    parsed_results.append(label)
    
parsed_data = [json.loads(item) for item in parsed_results]
df = pd.DataFrame(parsed_data)
df.columns = ['Model_label']

results = compute_metrics(df['Model_label'], test["PT_label"])
print(results)

error_analysis = pd.concat([test[["idx", "language", "MD_PT_label", "PT_label"]], df['Model_label']], axis=1)
error_analysis = error_analysis.rename(columns={'Model_label':"pred", "PT_label":"true"})
mask = error_analysis["pred"] == error_analysis["true"]
error_analysis = error_analysis[~ mask]
error_analysis.to_csv(f'PT_error_analysis_zero_shot_validation_{dataset}.csv', index=False)
print('-' * 100)
print('\n\n')


              precision    recall  f1-score   support

    negative       0.82      1.00      0.90         9
     neutral       1.00      0.25      0.40         4
    positive       0.67      1.00      0.80         2

    accuracy                           0.80        15
   macro avg       0.83      0.75      0.70        15
weighted avg       0.85      0.80      0.75        15

{'f1': 0.7533333333333333, 'precision': 0.8464646464646465, 'recall': 0.8}
----------------------------------------------------------------------------------------------------





### ICL approach
Below is code to evaluate the model with an ICL approach.
Update the right dataset and file path.

In [7]:
## CLI test
accelerator.wait_for_everyone()
start=time.time()

dataset = "90"


train = pd.read_csv(f"/sc/arion/projects/mscic1/psych_nlp/sentiment_analysis/PT_task/LLAMA3/3.1/best_CLI_sentences_{dataset}.csv")
test = pd.read_csv(f"/sc/arion/projects/mscic1/psych_nlp/sentiment_analysis/data/validation_sentences.csv")

#context sentences in json format
json_train_sentences = train["language"].to_json()
json_train_labels = train["PT_label"].to_json()

# converting test sentences to json format
json_test_sentences = test["language"].to_json()

# creating context prompt

SYSTEM_PROMPT = """
    <|start_header_id|>system<|end_header_id|>
    You are a patient at a medical center.<|eot_id|>
    """

USER_PROMPT_1 = f"""
    <|start_header_id|>user<|end_header_id|>
    Doctors write lots of clinical notes about you.
    Your task is to analyze the sentiment of a series of sentences your doctor wrote about you.
    For each sentence, how do you feel reading this description of you?
    Answer the question by assigning a sentiment score of negative, neutral, or positive for the sentence.
    Output your anser in JSON format. Don’t add explanation beyond the JSON.
    Below are some example sentences in JSON format:
    {json_train_sentences}<|eot_id|>
    """

ASSISTANT_PROMPT = f"""
    <|start_header_id|>assistant<|end_header_id|>
    f{json_train_labels}<|eot_id|>
    """

PROMPTS = []

for sentence in test["language"]:
    
    USER_PROMPT_2 = f"""
    <|start_header_id|>user<|end_header_id|>
    Complete the same task with this sentence and only return your sentiment score in JSON format:\n"{sentence}"<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """
    
    PROMPT = [f"""
    {SYSTEM_PROMPT}
    {USER_PROMPT_1}
    {ASSISTANT_PROMPT}
    {USER_PROMPT_2}
    """]
    PROMPTS.append(PROMPT)


with accelerator.split_between_processes(PROMPTS) as prompts:
    # store output of generations in dict   
    results=dict(outputs=[])

    # have each GPU do inference, prompt by prompt
    for prompt in prompts:
        prompt_tokenized=tokenizer(prompt, return_tensors="pt").to("cuda")
        output_tokenized = model.generate(**prompt_tokenized, max_new_tokens=10, do_sample=True, temperature=0.001, pad_token_id=tokenizer.eos_token_id)[0]

        # remove prompt from output
        output_tokenized=output_tokenized[len(prompt_tokenized["input_ids"][0]):]

        # store outputs and number of tokens in result{}
        results["outputs"].append( tokenizer.decode(output_tokenized) )
#         results["num_tokens"].append(len(output_tokenized))

        
    results=[ results ] # transform to list, otherwise gather_object() will not collect correctly

    # collect results from all the GPUs
results_gathered=gather_object(results)
# print(results_gathered)

results = results_gathered[0]['outputs']
parsed_results = []
for label in results:
    label = label.strip().replace('<|eot_id|>', '')
    parsed_results.append(label)
    
parsed_data = [json.loads(item) for item in parsed_results]
df = pd.DataFrame(parsed_data)
df.columns = ['Model_label']

results = compute_metrics(df['Model_label'], test["PT_label"])
print(results)

error_analysis = pd.concat([test[["idx", "language", "MD_PT_label", "PT_label"]], df['Model_label']], axis=1)
# print(error_analysis)
error_analysis = error_analysis.rename(columns={'Model_label':"pred", "PT_label":"true"})
mask = error_analysis["pred"] == error_analysis["true"]
error_analysis = error_analysis[~ mask]
error_analysis.to_csv(f'error_analysis_test_{dataset}.csv', index=False)
print('-' * 100)
print('\n\n')


              precision    recall  f1-score   support

    negative       0.75      1.00      0.86         9
     neutral       0.00      0.00      0.00         4
    positive       0.67      1.00      0.80         2

    accuracy                           0.73        15
   macro avg       0.47      0.67      0.55        15
weighted avg       0.54      0.73      0.62        15

{'f1': 0.6209523809523809, 'precision': 0.538888888888889, 'recall': 0.7333333333333333}
----------------------------------------------------------------------------------------------------





/hpc/packages/minerva-centos7/py_packages/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hpc/packages/minerva-centos7/py_packages/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/hpc/packages/minerva-centos7/py_packages/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

In [ ]:
# CLI experiment

accelerator.wait_for_everyone()
start=time.time()
# model.generation_config.temperature=None

params = {
    'seed': [42],
    'n_neutral_sentences': [0, 1, 2, 3, 4],
    'n_positive_sentences': [0, 1, 2, 3],
    'n_negative_sentences': [0, 1, 2, 3, 4]
}

dataset = "60"
metrics_file = f'PT_context_metrics_{dataset}.csv'
if os.path.isfile(metrics_file):
    f = open(metrics_file, 'a')
else:
    f = open(metrics_file, 'w')
    f.write('seed,n_neutral_sentences,n_positive_sentences,n_negative_sentences,f1,precision,recall\n')

best_model = []
best_f1 = 0.0
best_comb, best_results = None, None
for comb in list(ParameterGrid(params)):
    print(f"Testing with: {comb['n_negative_sentences']} negative, {comb['n_neutral_sentences']} neutral, {comb['n_positive_sentences']} positive.")
    train = pd.read_csv(f"/sc/arion/projects/mscic1/psych_nlp/sentiment_analysis/data/train_{dataset}.csv")
    test = pd.read_csv(f"/sc/arion/projects/mscic1/psych_nlp/sentiment_analysis/data/test_{dataset}.csv")
    # randomly selecting context sentences in json format
    train_sentences = train_sentence_selection(train,
                                               comb['n_neutral_sentences'],
                                               comb['n_positive_sentences'],
                                               comb['n_negative_sentences'],
                                               "PT",
                                               comb['seed'])
    json_train_sentences = train_sentences["language"].to_json()
    json_train_labels = train_sentences["PT_label"].to_json()

    # converting test sentences to json format
    json_test_sentences = test["language"].to_json()
    sentence_count = test['language'].size

    SYSTEM_PROMPT = """
    <|start_header_id|>system<|end_header_id|>
    You are a patient at a medical center.<|eot_id|>
    """

    USER_PROMPT_1 = f"""
    <|start_header_id|>user<|end_header_id|>
    Doctors write lots of clinical notes about you.
    Your task is to analyze the sentiment of a series of sentences your doctor wrote about you.
    For each sentence, how do you feel reading this description of you?
    Answer the question by assigning a sentiment score of negative, neutral, or positive for the sentence.
    Output your anser in JSON format. Don’t add explanation beyond the JSON.
    Below are some example sentences in JSON format:
    {json_train_sentences}<|eot_id|>
    """

    ASSISTANT_PROMPT = f"""
    <|start_header_id|>assistant<|end_header_id|>
    f{json_train_labels}<|eot_id|>
    """

    PROMPTS = []

    for sentence in test["language"]:

        USER_PROMPT_2 = f"""
        <|start_header_id|>user<|end_header_id|>
        Complete the same task with this sentence and only return your sentiment score in JSON format:\n"{sentence}"<|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """

        PROMPT = [f"""
        {SYSTEM_PROMPT}
        {USER_PROMPT_1}
        {ASSISTANT_PROMPT}
        {USER_PROMPT_2}
        """]
        PROMPTS.append(PROMPT)


    with accelerator.split_between_processes(PROMPTS) as prompts:
        # store output of generations in dict   
        results=dict(outputs=[])

        # have each GPU do inference, prompt by prompt
        for prompt in prompts:
            prompt_tokenized=tokenizer(prompt, return_tensors="pt").to("cuda")
            output_tokenized = model.generate(**prompt_tokenized, max_new_tokens=10, do_sample=True, temperature=0.001, pad_token_id=tokenizer.eos_token_id)[0]

            # remove prompt from output
            output_tokenized=output_tokenized[len(prompt_tokenized["input_ids"][0]):]

            # store outputs and number of tokens in result{}
            results["outputs"].append( tokenizer.decode(output_tokenized) )


        results=[ results ] # transform to list, otherwise gather_object() will not collect correctly

    # collect results from all the GPUs
    results_gathered=gather_object(results)
    results = results_gathered[0]['outputs']
    parsed_results = []
    for label in results:
        label = label.strip().replace('<|eot_id|>', '')
        parsed_results.append(label)

    parsed_data = [json.loads(item) for item in parsed_results]
    df = pd.DataFrame(parsed_data)
    df.columns = ['Model_label']

    results = compute_metrics(df['Model_label'], test["PT_label"])
    print(results)

    
    # saving results to metrics sheet
    v = [comb['seed'], comb['n_neutral_sentences'], comb['n_positive_sentences'], comb['n_negative_sentences'],
         results['f1'], results['precision'], results['recall']]
    f.write(','.join([str(el) for el in v]) + '\n')

    if results['f1'] > best_f1:
        best_f1 = results['f1']
        best_comb = comb
        best_results = results
        best_CLI_sentences = train_sentences
        best_CLI_sentences.to_csv(f'best_CLI_sentences_{dataset}.csv', index=False)
        error_analysis = pd.concat([test[["idx", "language", "MD_PT_label", "PT_label"]], df['Model_label']], axis=1)
    #   print(error_analysis)
        error_analysis = error_analysis.rename(columns={'Model_label':"pred", "PT_label":"true"})
        mask = error_analysis["pred"] == error_analysis["true"]
        error_analysis = error_analysis[~ mask]
        error_analysis.to_csv(f'error_analysis_validation_ICL_{dataset}.csv', index=False)
    print('-' * 100)
    print('\n\n')
    print('-' * 100)
    print('\n\n')

    if best_comb is not None:
        print(f'Best combination of context sentences: {best_comb}')
        print('\n')
        print(f'Best results: {best_results}')
    torch.cuda.empty_cache()
f.close()